## 1. 모델링

사전설정 (모델 훈련과 예측 모두 공통)

In [1]:
import os

model_save_path = "saved_model1"
# cols = ["KOSPI", "KOSPI_VOL", "CR", "GOLD", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI", "INDI", "FOREIGN", "ORG"]
# cols = ["KOSPI", "KOSPI_VOL", "CR", "GOLD", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI"]
cols = ["KOSPI", "KOSPI_VOL", "GOLD", "NASDAQ", "DOW", "NIKKEI"]
len_x_ARMA = 20
len_y_nextday = 20
scale_method = "norm"

# 패키지 캐시 모두 지우기 (tf 등 이전에 남은 가비지가 영향을 미칠 수 있음)
for dir_ in ["/Users/hrdkdh/Documents/GitHub/predict/", "/Users/hrdkdh/venv/"]:
    os.system("find {} | grep -E '(__pycache__|\.pyc$|\pyo$)' | xargs rm -rf".format(dir_))

1-1) 데이터 크롤링

In [2]:
from kospi_predict import Crawler

crawler = Crawler(crawl_page_max=15, perPage=100)
# crawler.crawlData(cols, save=True)
crawler.loadFromSavedFile(cols)
df_crawled = crawler.removeNan()
df_crawled

,date,KOSPI,KOSPI_VOL,GOLD,NASDAQ,DOW,NIKKEI
0,2022-03-02,2703.52,527950,1939.64,13752.016,33891.35,26393.030
1,2022-02-28,2699.18,613470,1900.55,13751.399,33892.60,26526.820
2,2022-02-25,2676.76,664347,1913.15,13694.624,34058.75,26476.500
3,2022-02-24,2648.80,926399,1945.00,13473.585,33223.83,25970.820
4,2022-02-23,2719.53,537841,1893.63,13037.487,33131.76,26210.215
...,...,...,...,...,...,...,...
1459,2016-03-24,1985.97,341173,1216.55,4773.505,17515.73,16892.330
1460,2016-03-23,1995.12,367551,1235.10,4768.861,17502.59,17000.980
1461,2016-03-22,1996.81,383752,1252.70,4821.659,17582.57,17048.550
1462,2016-03-21,1989.76,323260,1243.45,4808.872,17623.87,16886.680


1-2. 데이터 전처리

In [3]:
from kospi_predict import DataPreprocessor

dpp = DataPreprocessor(df_crawled, cols, scale_method, model_save_path)
dpp.sortByDate()
dpp.makeTargetXs(len_x_ARMA)
dpp.makeTargetYs(len_y_nextday)
dpp.cutoffData(len_x_ARMA, len_y_nextday)
dpp.scalingForModeling()
dpp.splitData()
dpp.df

,date,KOSPI,KOSPI_VOL,GOLD,NASDAQ,DOW,NIKKEI,X_KOSPI_DIFF_RATIO,X_KOSPI_VOL_DIFF_RATIO,X_GOLD_DIFF_RATIO,...,Y_KOSPI_nextday_11,Y_KOSPI_nextday_12,Y_KOSPI_nextday_13,Y_KOSPI_nextday_14,Y_KOSPI_nextday_15,Y_KOSPI_nextday_16,Y_KOSPI_nextday_17,Y_KOSPI_nextday_18,Y_KOSPI_nextday_19,Y_KOSPI_nextday_20
0,2016-04-19,-0.909430,401236,1242.58,4940.330,18053.60,16874.44,1.001125,1.182722,1.005853,...,-1.011159,-1.035033,-0.999426,-1.006885,-1.014878,-1.042800,-1.041918,-1.042991,-1.073063,-1.099704
1,2016-04-20,-0.923377,423964,1248.47,4948.130,18096.27,16906.54,0.997251,1.056645,1.004740,...,-1.033611,-0.997972,-1.005482,-1.013471,-1.041375,-1.040478,-1.041540,-1.071597,-1.098188,-1.097457
2,2016-04-21,-0.882342,392096,1258.10,4945.886,17982.52,17363.62,1.008111,0.924833,1.007713,...,-0.996552,-1.004027,-1.012067,-1.039965,-1.039054,-1.040100,-1.070140,-1.096719,-1.095941,-1.078312
3,2016-04-22,-0.899014,480470,1245.85,4906.228,18003.75,17572.49,0.996731,1.225389,0.990263,...,-1.002607,-1.010612,-1.038559,-1.037644,-1.038675,-1.068694,-1.095258,-1.094472,-1.076800,-1.122688
4,2016-04-25,-0.901384,397674,1233.05,4884.655,17873.56,17439.30,0.999534,0.827677,0.989726,...,-1.009191,-1.037102,-1.036238,-1.037265,-1.067267,-1.093806,-1.093011,-1.075334,-1.121168,-1.065027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,2022-01-21,1.166098,532753,1830.50,13768.922,34265.37,27522.26,0.990083,1.346461,0.994686,...,0.994977,0.932572,0.822193,0.750388,0.883287,0.918431,0.918511,0.915606,0.820994,0.851988
1419,2022-01-24,1.059437,473993,1840.85,13855.130,34364.50,27588.37,0.985079,0.889705,1.005654,...,0.933876,0.823507,0.751698,0.884473,0.919651,0.919516,0.916693,0.822163,0.853165,0.673349
1420,2022-01-25,0.878828,639958,1838.95,13539.295,34297.73,27131.34,0.974352,1.350142,0.998968,...,0.824818,0.753017,0.885775,0.920832,0.920736,0.917699,0.823267,0.854329,0.674557,0.743966
1421,2022-01-26,0.850707,472650,1845.38,13542.119,34168.09,27011.33,0.995901,0.738564,1.003497,...,0.754333,0.887084,0.922133,0.921917,0.918919,0.824294,0.855427,0.675749,0.745162,0.800591


In [4]:
#상관계수 보기
dpp.df_corr

,X,corr
0,KOSPI,0.982480
1,KOSPI_VOL,0.512137
2,GOLD,0.584337
3,NASDAQ,0.811459
4,DOW,0.786090
5,NIKKEI,0.868077


1-3) 모델 훈련하고 생성

In [5]:
from kospi_predict import ModelMaker

maker = ModelMaker(dpp.y_list, dpp.df_train, dpp.df_test, "relu", "RMSprop", model_save_path)
maker.makeModel(EPOCHS=400, history_plot_cutoff=30, learning_rate=0.001)

2022-03-03 21:55:23.669526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-03 21:55:23.669617: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-03 21:55:23.755193: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 246)               0         
                                                                 
 dense (Dense)               (None, 1476)              364572    
                                                                 
 dense_1 (Dense)             (None, 1476)              2180052   
                                                                 
 dense_2 (Dense)             (None, 1476)              2180052   
                                                                 
 dense_3 (Dense)             (None, 20)                29540     
                                                                 
Total params: 4,754,216
Trainable params: 4,754,216
Non-trainable params: 0
___________________________________

2022-03-03 21:55:23.956082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-03-03 21:55:24.622309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
maker.saveModelByCheckpoint("0198")
maker.validateModel(y_no=0)

In [ ]:
print(maker.test_mae)
print(maker.test_mse)

## 2. 최근 데이터로 20일 간의 KOSPI 지수 예측

2-1) 데이터 수집

In [ ]:
from kospi_predict import Crawler

#예측을 위해서는 현재기준 이전 10일간의 데이터만 있으면 됨 (len_x_ARMA가 10일때)
crawler_pred = Crawler(crawl_page_max=1, perPage=100) #이전 추세도 볼겸 넉넉히 수집
crawler_pred.crawlData(cols)
df_crawled_pred = crawler_pred.removeNan()
df_crawled_pred

2-2) 데이터 전처리

In [ ]:
from kospi_predict import DataPreprocessor

#아래는 예측을 위한 전처리
dpp_pred = DataPreprocessor(df_crawled_pred, cols, scale_method, model_save_path)
dpp_pred.sortByDate()
dpp_pred.makeTargetXs(len_x_ARMA)
dpp_pred.cutoffData(len_x_ARMA, 0)
dpp_pred.scalingForPredict()

2-3) 예측

In [ ]:
from kospi_predict import Predictor

#2.18 기준으로 20일 뒤까지 예측하고 시각화

predictor = Predictor(dpp_pred.df, scale_method = scale_method, model_save_path=model_save_path)
predictor.predict()
predictor.showPredictionPlot()

In [ ]:
len(predictor.x_cols)
predictor.saveModelToJS()